In [57]:
from IPython.display import HTML
import random
def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

In [58]:
from pydruid.client import *
from pydruid.utils.aggregators import *
from pydruid.db import connect
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import *
import random
import html

In [59]:


conn = connect(
  host='localhost',
  port=9999,
  path='druid/v2/sql',
  scheme='http')

curs = conn.cursor()

hide_toggle()



In [62]:
df = pd.DataFrame(curs.execute("""
    SELECT time_floor(__time,'PT1H') AS t1,
       sum("CO (ppm)") AS CO,
       sum("Humidity (%r.h.)") AS Humidity,
       sum("R1 (MOhm)") AS R1,
       sum("R3 (MOhm)") AS R3,
       sum("R4 (MOhm)") R4,
       sum("R14 (MOhm)") R14
FROM "druid"."regression_demo" 
GROUP BY 1
"""))

In [63]:
df

,t1,CO,Humidity,R1,R3,R4,R14
0,2016-09-30T20:00:00.000Z,30358,226527.3024,45507.6265,70439.0800,60419.1436,158794.6052
1,2016-09-30T21:00:00.000Z,165461,534345.0260,132414.8074,208537.4922,180201.3530,217264.3962
2,2016-09-30T22:00:00.000Z,79809,586290.1359,158090.1088,246551.3464,200512.8857,342610.5736
3,2016-09-30T23:00:00.000Z,70083,571926.4019,140637.4818,215127.7041,181558.1163,367726.1632
4,2016-10-01T00:00:00.000Z,130818,537379.5104,150811.8536,227893.2351,198065.0112,272620.8307
...,...,...,...,...,...,...,...
309,2016-10-17T02:00:00.000Z,114282,510180.4266,193325.8886,267148.2264,279541.0173,311295.7403
310,2016-10-17T03:00:00.000Z,105782,452243.7036,262425.8689,358317.8706,373927.3632,339167.6976
311,2016-10-17T04:00:00.000Z,45669,588479.8215,233266.5356,307558.7268,303627.5813,547745.4894
312,2016-10-17T05:00:00.000Z,220527,323496.4036,288353.7036,374042.9006,354024.9121,162097.0282


In [64]:
X = df[['Humidity','R1','R3','R4','R14']]
y = df[['CO']]

In [65]:
X

,Humidity,R1,R3,R4,R14
0,226527.3024,45507.6265,70439.0800,60419.1436,158794.6052
1,534345.0260,132414.8074,208537.4922,180201.3530,217264.3962
2,586290.1359,158090.1088,246551.3464,200512.8857,342610.5736
3,571926.4019,140637.4818,215127.7041,181558.1163,367726.1632
4,537379.5104,150811.8536,227893.2351,198065.0112,272620.8307
...,...,...,...,...,...
309,510180.4266,193325.8886,267148.2264,279541.0173,311295.7403
310,452243.7036,262425.8689,358317.8706,373927.3632,339167.6976
311,588479.8215,233266.5356,307558.7268,303627.5813,547745.4894
312,323496.4036,288353.7036,374042.9006,354024.9121,162097.0282


In [66]:
y

,CO
0,30358
1,165461
2,79809
3,70083
4,130818
...,...
309,114282
310,105782
311,45669
312,220527


In [67]:
from sklearn.model_selection import train_test_split

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [69]:
from sklearn.neighbors import KNeighborsRegressor
knnr = KNeighborsRegressor(n_neighbors = 50)
knnr.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=50)

In [70]:
print ("The MSE is:",format(np.power(y_test-knnr.predict(X_test),2).mean()))

The MSE is: CO    9.501070e+08
dtype: float64


In [71]:
df = pd.DataFrame(curs.execute("""
    SELECT time_floor(__time,'PT1M') AS t1,
       sum("CO (ppm)") AS CO,
       sum("Humidity (%r.h.)") AS Humidity,
       sum("R1 (MOhm)") AS R1,
       sum("R3 (MOhm)") AS R3,
       sum("R4 (MOhm)") R4,
       sum("R14 (MOhm)") R14
FROM "druid"."regression_demo" 
GROUP BY 1
"""))

In [72]:
X = df[['Humidity','R1','R3','R4','R14']]
y = df[['CO']]

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [78]:
knnr = KNeighborsRegressor(n_neighbors = 50)
knnr.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=50)

In [79]:
print ("The MSE is:",format(np.power(y_test-knnr.predict(X_test),2).mean()))

The MSE is: CO    197303.68953
dtype: float64


In [80]:
df = pd.DataFrame(curs.execute("""
    SELECT time_floor(__time,'PT1S') AS t1,
       avg("CO (ppm)") AS CO,
       avg("Humidity (%r.h.)") AS Humidity,
       avg("R1 (MOhm)") AS R1,
       avg("R3 (MOhm)") AS R3,
       avg("R4 (MOhm)") R4,
       avg("R14 (MOhm)") R14
FROM "druid"."regression_demo" where __time between '2016-09-30' and '2016-10-08'
GROUP BY 1
"""))

In [82]:
X = df[['Humidity','R1','R3','R4','R14']]
y = df[['CO']]

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [86]:
knnr = KNeighborsRegressor(n_neighbors = 10)
knnr.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=10)

In [87]:
print ("The MSE is:",format(np.power(y_test-knnr.predict(X_test),2).mean()))

The MSE is: CO    11.576547
dtype: float64


In [88]:
df = pd.DataFrame(curs.execute("""
    SELECT __time AS t1,
       avg("CO (ppm)") AS CO,
       avg("Humidity (%r.h.)") AS Humidity,
       avg("R1 (MOhm)") AS R1,
       avg("R3 (MOhm)") AS R3,
       avg("R4 (MOhm)") R4,
       avg("R14 (MOhm)") R14
FROM "druid"."regression_demo" where __time between '2016-09-30' and '2016-10-08'
GROUP BY 1
"""))

In [89]:
X = df[['Humidity','R1','R3','R4','R14']]
y = df[['CO']]

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [91]:
knnr = KNeighborsRegressor(n_neighbors = 10)
knnr.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=10)

In [92]:
print ("The MSE is:",format(np.power(y_test-knnr.predict(X_test),2).mean()))

The MSE is: CO    8.891102
dtype: float64


In [93]:
df = pd.DataFrame(curs.execute("""
    SELECT __time AS t1,
       avg("CO (ppm)") AS CO,
       avg("Humidity (%r.h.)") AS Humidity,
       avg("R1 (MOhm)") AS R1,
       avg("R3 (MOhm)") AS R3,
       avg("R4 (MOhm)") R4,
       avg("R14 (MOhm)") R14
FROM "druid"."regression_demo" where __time between '2016-09-30' and '2016-10-03'
GROUP BY 1
"""))

In [94]:
X = df[['Humidity','R1','R3','R4','R14']]
y = df[['CO']]

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [96]:
knnr = KNeighborsRegressor(n_neighbors = 10)
knnr.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=10)

In [97]:
print ("The MSE is:",format(np.power(y_test-knnr.predict(X_test),2).mean()))

The MSE is: CO    8.907165
dtype: float64
